In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow.keras.optimizers.legacy import Adam

import tensorflow_privacy
# from tensorflow_privacy.privacy.analysis import compute_dp_sgd_privacy
from tensorflow_privacy.privacy.optimizers import dp_optimizer_keras
from tensorflow_privacy.privacy.optimizers import dp_optimizer
from tensorflow_privacy.privacy.optimizers import dp_optimizer_vectorized
from tensorflow_privacy.privacy.optimizers import dp_optimizer_keras_vectorized
from tensorflow_privacy.privacy.analysis import compute_dp_sgd_privacy

from synthesizers.cgan.model import ConditionalGAN, GANMonitor
from synthesizers.preprocessing.wesad import Subject, WESADDataset
from synthesizers.utils.training import data_split, buildDataCTST, generate_and_plot_data, synthetic_dataset


In [ ]:
DATA_PATH = 'data/wesad/wesad_preprocessed_1hz.csv'
SAMPLING_RATE = 1

In [ ]:
df = pd.read_csv(DATA_PATH, index_col=0)
df_label = df['label']
df_stress = df[df['label']==1]
df_no_stress = df[df['label']==0]

In [ ]:
trainX, label_trainX = WESADDataset.create_windows(df, 1)
mos, _ = WESADDataset.create_windows(df_stress, SAMPLING_RATE)
non_mos, _ = WESADDataset.create_windows(df_no_stress,SAMPLING_RATE)

trainX = np.delete(trainX, 6, axis=2)
mos = np.delete(mos, 6, axis=2)
non_mos = np.delete(non_mos, 6, axis=2)

In [ ]:
num_split = 0.8
trainmos, testmos = data_split(mos, num_split)
trainnomos, testnomos = data_split(non_mos, num_split)

print(trainmos.shape)
print(testmos.shape)
print(trainnomos.shape)
print(testnomos.shape)

In [ ]:
# Training Hyperparameters
DP_TRAINING = False
NUM_FEATURES = trainX.shape[2]
SEQ_LENGTH = 60
LATENT_DIM = SEQ_LENGTH
BATCH_SIZE = 32
HIDDEN_UNITS = 16
EPOCHS = 30000

# load dataset into tf dataset
dataset = tf.data.Dataset.from_tensor_slices((trainX, label_trainX))
dataset = dataset.shuffle(buffer_size=1024).batch(BATCH_SIZE)
dataset

In [ ]:
#tf.random.set_seed(42)
randomTrainMos = tf.random.normal(shape=(trainmos.shape[0], LATENT_DIM))

#tf.random.set_seed(42)
randomTrainNoMos = tf.random.normal(shape=(trainnomos.shape[0], LATENT_DIM))

#tf.random.set_seed(42)
randomTestMos = tf.random.normal(shape=(testmos.shape[0], LATENT_DIM))

#tf.random.set_seed(42)
randomTestNoMos = tf.random.normal(shape=(testnomos.shape[0], LATENT_DIM))

In [ ]:
l2_norm_clip = 1.5
noise_multiplier = 1.3
num_microbatches = 250
learning_rate = 0.25

optimizer = dp_optimizer_vectorized.VectorizedDPAdamOptimizer( #vectorized adam am schnellsten
    l2_norm_clip=l2_norm_clip,
    noise_multiplier=noise_multiplier,
    num_microbatches=num_microbatches,
    learning_rate=learning_rate
)
# optimizer = dp_optimizer_keras_vectorized.VectorizedDPKerasAdamOptimizer(
#     l2_norm_clip=l2_norm_clip,
#     noise_multiplier=noise_multiplier,
#     num_microbatches=num_microbatches,
#     learning_rate=learning_rate
# )


In [ ]:
cond_gan = ConditionalGAN(
    num_features=NUM_FEATURES,
    seq_length=SEQ_LENGTH,
    latent_dim=LATENT_DIM,
    discriminator=ConditionalGAN.conditional_discriminator(
        seq_length=SEQ_LENGTH, 
        num_features=NUM_FEATURES), 
    generator=ConditionalGAN.conditional_generator(
        hidden_units=HIDDEN_UNITS, 
        seq_length=SEQ_LENGTH, 
        latent_dim=LATENT_DIM,
        num_features=NUM_FEATURES
    )
)
if DP_TRAINING:
    l2_norm_clip = 1.5
    noise_multiplier = 1.3
    num_microbatches = 250
    learning_rate = 0.25

    d_optimizer = dp_optimizer_vectorized.VectorizedDPAdamOptimizer( #vectorized adam am schnellsten
        l2_norm_clip=l2_norm_clip,
        noise_multiplier=noise_multiplier,
        num_microbatches=num_microbatches,
        learning_rate=learning_rate
    )
else:
    d_optimizer = Adam(learning_rate=0.0002, beta_1=0.5)

cond_gan.compile(
    d_optimizer= d_optimizer,# Adam(learning_rate=0.0002, beta_1=0.5),
    g_optimizer= Adam(learning_rate=0.0002, beta_1=0.5), #optimizer
    loss_fn=tf.keras.losses.BinaryCrossentropy(from_logits=True),
)

print(f"{cond_gan.d_optimizer} is used")

if DP_TRAINING:
    generator_save_path = f"models/dp/{EPOCHS}/cgan_generator/"
else:
    generator_save_path = f"models/no_dp/{EPOCHS}/cgan_generator"

history = cond_gan.fit(
    dataset,
    epochs=EPOCHS,
    callbacks=[
        GANMonitor(
            trainmos,
            trainnomos,
            testmos,
            testnomos,
            randomTrainMos,
            randomTrainNoMos,
            randomTestMos,
            randomTestNoMos,
            num_seq=50,
            save_path=generator_save_path,
            #dp=True,
            #noise_multiplier=noise_multiplier,
            batch_size=BATCH_SIZE,
            seq_length=SEQ_LENGTH,
            num_features=NUM_FEATURES,
        )
    ],
)

if DP_TRAINING:
    base_path = f"models/dp/{EPOCHS}/"
    cond_gan.generator.save(f"{base_path}cgan_generator")
    cond_gan.discriminator.save(f"{base_path}cgan_discriminator")
else:
    base_path = f"models/no_dp/{EPOCHS}/"
    cond_gan.generator.save(f"{base_path}cgan_generator")
    cond_gan.discriminator.save(f"{base_path}cgan_discriminator")


In [ ]:
compute_dp_sgd_privacy.compute_dp_sgd_privacy(n=trainX.shape[0],
                                              batch_size=32,
                                              noise_multiplier=noise_multiplier,
                                              epochs=10,
                                              delta=1e-5)
     
